In [ ]:
cd ~/MultiFidelity-ProcessOpt/Perovskites/

In [ ]:
#!/usr/bin/env python

import json
import pickle
import numpy as np
import pandas as pd
from copy import deepcopy

from olympus.datasets import Dataset
from olympus.objects import (
	ParameterContinuous,
	ParameterDiscrete, 
	ParameterCategorical, 
	ParameterVector
)
from olympus.campaigns import ParameterSpace, Campaign

from atlas.planners.multi_fidelity.planner import MultiFidelityPlanner



In [ ]:
# config
dataset = Dataset(kind='perovskites')
NUM_RUNS = 2
# BUDGET = 30
COST_BUDGET = 50 # 200.
NUM_INIT_DESIGN = 10
NUM_CHEAP = 8

# lookup table
# organic --> cation --> anion --> bandgap_hse06/bandgap_gga
LOOKUP = pickle.load(open('0.Data/lookup_table.pkl', 'rb'))
# print(lookup.keys())
# print(lookup['Ethylammonium']['Ge']['F'].keys())



In [ ]:

def build_options_descriptors(json_file):
	with open(json_file, 'r') as f:
		content = json.load(f)

	options = list(content.keys())
	descriptors = [list(content[opt].values()) for opt in options]

	return options, descriptors

def measure(params, s):
	# high-fidelity is hse06, low-fidelity is gga
	if s == 1.0:
		measurement = np.amin(
			LOOKUP[params.organic.capitalize()][params.cation][params.anion]['bandgap_hse06']
		)
	elif s == 0.1:
		measurement = np.amin(
			LOOKUP[params.organic.capitalize()][params.cation][params.anion]['bandgap_gga']
		)
	return measurement

def get_min_hse06_bandgap(param_space):
	organic_options = [o.capitalize() for o in param_space[1].options]
	cation_options = [o.capitalize() for o in param_space[2].options]
	anion_options = [o.capitalize() for o in param_space[3].options]

	hse06_bandgaps = []
	for organic_option in organic_options:
		for cation_option in cation_options:
			for anion_option in anion_options:
				hse06_bandgaps.append(
					np.amin(
						LOOKUP[organic_option][cation_option][anion_option]['bandgap_hse06']
					)
				)
	min_hse06_bandgap = np.amin(hse06_bandgaps)
	return min_hse06_bandgap

def compute_cost(params):
	costs = params[:,0].astype(float)
	return np.sum(costs)



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from copy import deepcopy

class TransferLearningDNN:
    def __init__(self, input_dim, hidden_dim=64, device='cpu'):
        self.model = self._build_model(input_dim, hidden_dim).to(device)
        self.input_dim = input_dim
        self.device = device
        self.scaler = None  # feature normalization (optional)

    def _build_model(self, input_dim, hidden_dim):
        # 심플한 2층 DNN
        return nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def fit_scaler(self, X):
        # min-max scaler
        self.x_min = X.min(axis=0)
        self.x_max = X.max(axis=0)

    def pretrain(self, X_low, y_low, epochs=300, lr=1e-3, verbose=False):
        # low-fidelity 데이터로 선학습
        X_low, y_low = np.asarray(X_low), np.asarray(y_low).flatten()
        if self.scaler is None:
            self.fit_scaler(X_low)
        X_low = self.transform(X_low)
        X_tensor = torch.tensor(X_low, dtype=torch.float32).to(self.device)
        y_tensor = torch.tensor(y_low, dtype=torch.float32).to(self.device)
        optimizer = optim.Adam(self.model.parameters(), lr=lr)
        loss_fn = nn.MSELoss()
        self.model.train()
        for epoch in range(epochs):
            optimizer.zero_grad()
            pred = self.model(X_tensor).squeeze()
            loss = loss_fn(pred, y_tensor)
            loss.backward()
            optimizer.step()
            if verbose and (epoch+1) % 50 == 0:
                print(f'[Pretrain] Epoch {epoch+1}: Loss {loss.item():.4f}')

    def finetune(self, X_high, y_high, epochs=100, lr=1e-4, verbose=False):
        # high-fidelity 데이터로 파인튜닝
        X_high, y_high = np.asarray(X_high), np.asarray(y_high).flatten()
        X_tensor = torch.tensor(X_high, dtype=torch.float32).to(self.device)
        y_tensor = torch.tensor(y_high, dtype=torch.float32).to(self.device)
        optimizer = optim.Adam(self.model.parameters(), lr=lr)
        loss_fn = nn.MSELoss()
        self.model.train()
        for epoch in range(epochs):
            optimizer.zero_grad()
            pred = self.model(X_tensor).squeeze()
            loss = loss_fn(pred, y_tensor)
            loss.backward()
            optimizer.step()
            if verbose and (epoch+1) % 20 == 0:
                print(f'[Finetune] Epoch {epoch+1}: Loss {loss.item():.4f}')

    def predict(self, X):
        # (GA 등에서 호출) 입력 X에 대해 예측값 리턴
        X = np.asarray(X)
        X_tensor = torch.tensor(X, dtype=torch.float32).to(self.device)
        self.model.eval()
        with torch.no_grad():
            y_pred = self.model(X_tensor).cpu().numpy().flatten()
        return y_pred

    def get_fitness_func(self, y_best=None, xi=0.01):
        # pymoo 같은 GA에서 쓰기 위한 objective 함수 래퍼
        # 여기선 EI(예상 개선량) 형태로 예시 구현
        def fitness(x):
            mu = self.predict(np.array([x]))[0]
            if y_best is not None:
                return -(y_best - mu - xi)  # maximize EI → minimize -EI
            else:
                return mu  # 단순 예측값 최적화
        return fitness
